In [228]:
import pandas as pd
import re
import numpy as np

In [229]:
df = pd.read_csv("day10.txt", header=None)
df.columns = ["coding"]
df

,coding
0,7-F7-
1,.FJ|7
2,SJLL7
3,|F--J
4,LJ.LJ


In [230]:
x_dim = df.coding.str.len()[0]
y_dim = len(df)
x_dim, y_dim

(5, 5)

In [231]:
symbols = np.zeros((x_dim, y_dim), dtype="str")

In [232]:
starting_node = None

In [233]:
numbers = []
for y, row in enumerate(df.coding):
    for x, val in enumerate(row):
        symbols[y, x] = val
        if val == "S":
            starting_node = y, x
symbols

array([['7', '-', 'F', '7', '-'],
       ['.', 'F', 'J', '|', '7'],
       ['S', 'J', 'L', 'L', '7'],
       ['|', 'F', '-', '-', 'J'],
       ['L', 'J', '.', 'L', 'J']], dtype='<U1')

In [234]:
starting_node

(2, 0)

In [235]:
visited_nodes = {starting_node: 0}

In [236]:
from collections import deque
queue = deque()
queue.append(starting_node)

In [237]:
class SymbolDefinition:
    def __init__(self, symbol, new_locations_generator) -> None:
        self.symbol = symbol
        self.new_locations_generator = new_locations_generator

    def get_new_locations(self, location):
        y, x = location
        return self.new_locations_generator(y, x)

In [238]:
definitions = [
SymbolDefinition("-", lambda y, x: [(y, x-1), (y, x+1)]),
SymbolDefinition("|", lambda y, x: [(y+1, x), (y-1, x)]),
SymbolDefinition("L", lambda y, x: [(y, x+1), (y-1, x)]),
SymbolDefinition("J", lambda y, x: [(y, x-1), (y-1, x)]),
SymbolDefinition("7", lambda y, x: [(y, x-1), (y+1, x)]),
SymbolDefinition("F", lambda y, x: [(y, x+1), (y+1, x)])
]

In [239]:
symbol_def_map = {symboldef.symbol: symboldef for symboldef in definitions}

In [240]:
def process_locations(new_locations, prev_x, prev_y):
    current_steps = visited_nodes[(prev_y, prev_x)]
    for new_loc in new_locations:
        if new_loc not in visited_nodes:
            visited_nodes[new_loc] = current_steps + 1
            queue.append(new_loc)

while len(queue):
    current_search = queue.popleft()
    y, x = current_search
    #print(f"processing {current_search}")
    symbol = symbols[y, x]
    if symbol == "S":
        if symbols[y, x+1] == "-":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y, x-1] == "-":
            visited_nodes[(y, x-1)] = 1
            queue.append((y, x-1))
        if symbols[y+1, x] == "|":
            visited_nodes[(y+1, x)] = 1
            queue.append((y+1, x))
        if symbols[y-1, x] == "|":
            visited_nodes[(y-1, x)] = 1
            queue.append((y-1, x))
        if symbols[y-1, x] == "J":
            visited_nodes[(y-1, x)] = 1
            queue.append((y-1, x))
        if symbols[y, x+1] == "J":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y+1, x] == "L":
            visited_nodes[(y+1, x)] = 1
            queue.append((y+1, x))
        print(queue)
    else:
        symbol_def = symbol_def_map[symbol]
        new_locations = symbol_def.get_new_locations(current_search)
        process_locations(new_locations, x, y)   



deque([(3, 0), (2, 1)])


In [241]:
visited_nodes

{(2, 0): 0,
 (3, 0): 1,
 (2, 1): 1,
 (4, 0): 2,
 (1, 1): 2,
 (4, 1): 3,
 (1, 2): 3,
 (3, 1): 4,
 (0, 2): 4,
 (3, 2): 5,
 (0, 3): 5,
 (3, 3): 6,
 (1, 3): 6,
 (3, 4): 7,
 (2, 3): 7,
 (2, 4): 8}